In [94]:
from selenium import webdriver 
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
from time import time, sleep
import os
import datetime 
import json 
import re

In [95]:
# selenium 브라우저를 숨긴채로 작업할것인지 아닌지
# 디버그를 위해 브라우저를 숨기지 않고 작업한다.
options = webdriver.ChromeOptions()
options.add_argument('window-size=1024x648')
hidden_browser = False
if hidden_browser:
    options.add_argument('headless')
    
    options.add_argument("disable-gpu")
    
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
driver = webdriver.Chrome(executable_path='/Users/dooyeoung/Documents/projects/webdriver/chromedriver', 
                        chrome_options=options,
                        desired_capabilities=capa)

# 특정 개체를 로딩 될때까지 대기하기 위해 필요하다
wait = WebDriverWait(driver, 20)

#  카페24 로그인
driver.get('https://eclogin.cafe24.com/Shop/')

# id 입력 input이 로딩될대까지 기다린후 id와 password 입력
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#mall_id')))

driver.execute_script('''
document.getElementById('mall_id').value = 'instanttattoo';
document.getElementById('userpasswd').value = 'namdosan2021!';
''')

driver.find_element_by_css_selector('#tabAdmin > div > fieldset > p.gButton > a').click()

# cafe24 상위 메뉴가 로딩될때까지 기다린후 
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#gnb'))) 
# 일별 매출 조회 페이지로 이동
driver.get('https://instanttattoo.cafe24.com/disp/admin/shop1/report/DailyList')

In [97]:
# 자세히보기 체크박스 클릭
driver.find_element_by_id('sReportGabView').click()

sleep(1)
# 기간설정 input이 로드 될때까지 기다린후 
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#pr_start_date'))) 

sleep(1)
# 매출 데이터를 수집할 기간을 지정
start_date = datetime.datetime(2020, 1, 1)
end_edate = datetime.datetime(2020,12, 15) 
start_date_str = start_date.strftime('%Y-%m-%d')
end_edate_str = end_edate.strftime('%Y-%m-%d')

# 기간설정과 
driver.execute_script('''
document.getElementById('pr_start_date').setAttribute('value', '{}');
document.getElementById('pr_end_date').setAttribute('value', '{}');

document.querySelector('select#rows option[value="100"]').setAttribute('value', 500);
'''.format(start_date_str, end_edate_str))

driver.find_element_by_css_selector('select#rows option[value="500"]').click()

In [99]:
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.mBoard.gScroll > table'))) 
sleep(1)

# 데이터 테이블 태그를 빠르게 파싱하기 위해 BeautifulSoup을 사용한다.
table_tag = driver.find_element_by_css_selector('.mBoard.gScroll > table')
soup = BeautifulSoup(table_tag.get_attribute('outerHTML'), 'lxml')
row_li = []
for tr in soup.select('tbody > tr'):
    td_li = tr.select('td')
    row = {
        "date" : td_li[0].text,
        "주문수" : td_li[1].text,
        "품목수" : td_li[2].text,
        "상품구매금액" : td_li[3].text,
        "배송비" : td_li[4].text,
        "할인" : td_li[5].text,
        "쿠폰" : td_li[6].text,
        "실제결제금액" : td_li[7].text,
        "적립금" : td_li[8].text,
        "예치금" : td_li[9].text,
        "네이버포인트" : td_li[10].text,
        "결제합계" : td_li[11].text,
    }
    row_li.append(row)
    
print( len(row_li))

# 수집된 데이터를 확인해본다. 잘 수집되었다.
rdf = pd.DataFrame(row_li)

350


In [101]:
rdf.head(20)

,date,주문수,품목수,상품구매금액,배송비,할인,쿠폰,실제결제금액,적립금,예치금,네이버포인트,결제합계
0,2020-12-15(화),36,86,"1,572,480","48,000","32,840","4,000","1,223,199","11,328",0,"349,113","1,583,640"
1,2020-12-14(월),31,70,"1,224,100","36,000",0,"8,000","1,130,726","15,958",0,"105,416","1,252,100"
2,2020-12-13(일),15,36,"535,400","12,000",0,"8,000","315,955",0,0,"223,445","539,400"
3,2020-12-12(토),17,45,"647,100","27,000","7,000",0,"528,658",0,0,"138,442","667,100"
4,2020-12-11(금),24,50,"1,886,000","33,000","10,400","4,000","1,855,378","27,522",0,"21,700","1,904,600"
5,2020-12-10(목),42,82,"1,898,500","36,000","9,100",0,"1,558,714","1,416",0,"365,270","1,925,400"
6,2020-12-09(수),68,162,"3,097,100","54,000","8,980",0,"2,749,350",0,0,"392,770","3,142,120"
7,2020-12-08(화),114,230,"5,658,200","93,000","11,640",0,"5,280,249","63,724",0,"395,587","5,739,560"
8,2020-12-07(월),311,649,"11,845,400","357,000","57,240",0,"11,071,754","21,410",0,"1,051,996","12,145,160"
9,2020-12-06(일),2,4,"27,600","9,000","3,540",0,"23,060",0,0,"10,000","33,060"
